### Infinite Push

Given a dataset $X_p = \{x^{+}_1,...,x^{+}_m\}$ of data vectors classified as "relevant" and $X_n = \{x^{-}_1,...,x^{-}_n\}$ of data vectors classified as "irrelevant", we would like to come up with a linear ranking function $f(x) = w^Tx$ that maximizes the number of relevant vectors in $X_p$ ranked above the highest ranking irrelevant vector. Another way of saying this is that we would like to minimize the maximum number of relevant vectors ranked below any irrelevant vector. In other words, we would like to minimize the function:

\begin{equation}
\max_{1 \leq j \leq n} \sum_{i = 1}^m \mathbb{1}[f(x^{+}_i) < f(x^{-}_j)]
\end{equation}

where $\mathbb{1}$ is the indicator function that is 1 when its argument is true and false otherwise.

Since this function is not convex (the indicator function is discontinuous) we replace the indicator function with its hinge loss convex relaxation:

\begin{equation}
\max_{1 \leq j \leq n} \sum_{i = 1}^m \max(1 - (f(x^{+}_i) - f(x^{-}_j)), 0)
\end{equation}

\begin{equation}
\max_{1 \leq j \leq n} \sum_{i = 1}^m \max(1 - w^T(x^{+}_i - x^{-}_j), 0)
\end{equation}

This is a pointwise maximum of affine functions of $w$ and is therefore convex in $w$.

We add an $\ell_2$ regularization term to form the final problem:
\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \max_{1 \leq j \leq n} \sum_{i = 1}^m \max(1 - w^T(x^{+}_i - x^{-}_j), 0) + \lambda \|w|_2^2 \\
  \end{aligned}
\end{equation*}

with variable $w$, regularization parameter $\lambda$, and constants $x^{+}_i$ for $i = 1,...,m$ and $x^{-}_j$ for $j = 1,...,n$.

References: Agarwal, Shivani 2011. "The infinite push: A new support vector ranking algorithm that directly optimizes accuracy at the absolute top of the list"

In [1]:
import cvxpy as cp
import numpy as np
import scipy as sp

# setup

problemID = "infinite_push_0"
prob = None
opt_val = None

# Variable declarations


def normalized_data_matrix(m, n, mu):
    if mu == 1:
        # dense
        A = np.random.randn(m, n)
        A /= np.sqrt(np.sum(A**2, 0))
    else:
        # sparse
        A = sps.rand(m, n, mu)
        A.data = np.random.randn(A.nnz)
        N = A.copy()
        N.data = N.data**2
        A = A*sps.diags([1 / np.sqrt(np.ravel(N.sum(axis=0)))], [0])

    return A

m = 100
n = 200
d = 20
np.random.seed(0)

Xp = normalized_data_matrix(m, d, 1)
Xn = normalized_data_matrix(n, d, 1)
lam = 1


# Problem construction

def infinite_push(theta, Xp, Xn):
    m, d = Xp.shape
    n = Xn.shape[0]
    Z = cp.max_elemwise(
        1 - (Xp*theta*np.ones((1,n)) - (Xn*theta*np.ones((1,m))).T), 0)
    return cp.max_entries(cp.sum_entries(Z, axis=0))

theta = cp.Variable(d)
f = infinite_push(theta, Xp, Xn) + lam*cp.sum_squares(theta)
prob = cp.Problem(cp.Minimize(f))


# Problem collection

# Single problem collection
problemDict = {
    "problemID" : problemID,
    "problem"   : prob,
    "opt_val"   : opt_val
}
problems = [problemDict]



# For debugging individual problems:
if __name__ == "__main__":
    def printResults(problemID = "", problem = None, opt_val = None):
        print(problemID)
        problem.solve()
        print("\tstatus: {}".format(problem.status))
        print("\toptimal value: {}".format(problem.value))
        print("\ttrue optimal value: {}".format(opt_val))
    printResults(**problems[0])




infinite_push_0
	status: optimal
	optimal value: 100.0
	true optimal value: None
